In [ ]:
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.


### Data Processing

In [ ]:
import os
import cv2
import numpy as np

!pip install yt_dlp

import yt_dlp

import matplotlib.pyplot as plt

def download_youtube_video(url, output_path):
    ydl_opts = {
        'format': 'bestvideo[height=1080][ext=mp4]/mp4',
        'outtmpl': output_path,
    }
    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        ydl.download([url])
    return output_path

def video_to_3d_data_frame(video_path, segment_number):
    video_data = []
    start_frame = (segment_number - 1) * 3000
    end_frame = segment_number * 3000
    cap = cv2.VideoCapture(video_path)

    if not cap.isOpened():
        print("Error: Could not open video.")
        return None

    cap.set(cv2.CAP_PROP_POS_FRAMES, start_frame)  # move the current position to the start frame

    current_frame = start_frame
    while current_frame < end_frame and cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        video_data.append(rgb_frame)
        current_frame += 1

    cap.release()
    video_data_array = np.array(video_data)
    return video_data_array

def yt_to_data(url, segment_number):
    youtube_url = url
    video_path = 'downloaded_video.mp4'
    downloaded_video_path = download_youtube_video(youtube_url, video_path)
    video_data_array = video_to_3d_data_frame(downloaded_video_path, segment_number)

    #os.remove(downloaded_video_path)  # ensure the file is removed after processing

    return video_data_array


train_url1 = 'https://www.youtube.com/watch?v=j0HoMaaQj9I'
train_url2 = 'https://www.youtube.com/watch?v=1jzJGcRdxPY'
train_url3 = 'https://www.youtube.com/watch?v=Bs58RoTf-g8'
train_url4 = 'https://www.youtube.com/watch?v=XoyYtqi5u54'
train_url5 = 'https://www.youtube.com/watch?v=rMPkUuMq024'
train_url6 = 'https://www.youtube.com/watch?v=JUWtWPX6hgs'
train_url7 = 'https://www.youtube.com/watch?v=WtnbT6ft710'
train_url8 = 'https://www.youtube.com/watch?v=QLF0FXcW25E'

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 69.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.4/194.4 kB 14.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 60.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 8.4 MB/s eta 0:00:00


### Setting up Training Pairs for Data

Keep in mind how many iterations are necessary for a samples of 3000 frames per video. You can calculate the total number of frames by looking at the 'stats for nerds' setting on the YouTube video and multiplying the FPS by the length of the video. Then divide that by 3000 to determine the number of iterations necessary. Then add that to the urls dictionary

In [ ]:
import torch
from torch.nn import functional

def create_training_pairs(video_data_array):
    print("Starting the Numpy!")
    length = len(video_data_array) - 2
    # pre-allocate tensors
    input_frames = torch.zeros((length // 2, 2, *video_data_array.shape[1:]), dtype=video_data_array.dtype)
    target_frames = torch.zeros((length // 2, *video_data_array.shape[1:]), dtype=video_data_array.dtype)

    print("Entering loop")
    for i in range(0, length - 1, 2):
        input_frames[i // 2] = video_data_array[i:i+2]  # two consecutive frames
        target_frames[i // 2] = video_data_array[i+1]   # middle frame
        if i % 100 == 0:
          print(i, "/", length, flush=True)
    print("Finished loop")
    return input_frames, target_frames

urls = {train_url1 : 1,
        train_url2 : 1,
        train_url3 : 5,
        train_url4 : 2,
        train_url5 : 1,
        train_url6 : 6,
        train_url7 : 1,
        train_url8 : 11
        }

### Creating the CNN Model

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

Using cuda device


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset, TensorDataset, ConcatDataset

class FrameInterpolationCNN(nn.Module):
    def __init__(self):
        super(FrameInterpolationCNN, self).__init__()
        # Encoding layers for individual frames
        self.encoder = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, stride=2, padding=1),  # downsample
            nn.ReLU(),
            nn.Conv2d(64, 128, kernel_size=3, stride=2, padding=1),
            nn.ReLU(),
            nn.Conv2d(128, 256, kernel_size=3, stride=2, padding=1),
            nn.ReLU()
        )
        # Decoding layers to interpolate the frame
        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(256 * 2, 128, kernel_size=3, stride=2, padding=1, output_padding=1), # upsample
            nn.ReLU(),
            nn.ConvTranspose2d(128, 64, kernel_size=3, stride=2, padding=1, output_padding=1),
            nn.ReLU(),
            nn.ConvTranspose2d(64, 3, kernel_size=3, stride=2, padding=1, output_padding=1),
            nn.Sigmoid()
        )

    def forward(self, frame1, frame2):
        # Encode both frames separately
        enc1 = self.encoder(frame1)
        enc2 = self.encoder(frame2)

        # Concatenate the encoded frames
        enc = torch.cat((enc1, enc2), dim=1)  # concatenate along the channel dimension

        # Decode to get the interpolated frame
        out = self.decoder(enc)
        return out

class FrameDataset(Dataset):
    def __init__(self, input_frames, target_frames):
        self.input_frames = input_frames
        self.target_frames = target_frames

    def __len__(self):
        return len(self.target_frames)

    def __getitem__(self, idx):
        # normalize pixel data to range [0, 1]
        frame1 = self.input_frames[idx][0].float() / 255.0
        frame2 = self.input_frames[idx][1].float() / 255.0
        target_frame = self.target_frames[idx].float() / 255.0

        # permute the dimensions to [C, H, W]
        frame1 = frame1.permute(2, 0, 1)
        frame2 = frame2.permute(2, 0, 1)
        target_frame = target_frame.permute(2, 0, 1)

        return (frame1, frame2), target_frame


model = FrameInterpolationCNN().to(device)

criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)

num_epochs = 10

train_array = None
input_frames, target_frames = None, None

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    counter = 1
    for url in urls:
      for j in range(urls[url]):

        train_array = yt_to_data(url, j+1)
        train_array = torch.from_numpy(train_array)
        input_frames, target_frames = create_training_pairs(train_array)
        train_array = None
        frame_dataset = FrameDataset(input_frames, target_frames)
        train_loader = DataLoader(frame_dataset, batch_size=10, shuffle=False, num_workers=4) # I ran it on 12 CPU Cores, use GPU if you can

        for i, ((frame1, frame2), target_frame) in enumerate(train_loader):
          frame1, frame2, target_frame = frame1.to(device), frame2.to(device), target_frame.to(device)

          # Zero the parameter gradients
          optimizer.zero_grad()

          # Forward pass
          outputs = model(frame1, frame2)

          # Compute loss
          loss = criterion(outputs, target_frame)

          # Backward pass and optimize
          loss.backward()
          optimizer.step()

          # print statistics so that we can tell the progress of this model as it runs
          running_loss += loss.item()
          if i % 10 == 9:    # print every 10 mini-batches
              print(f'Url: {counter}, Sample: {j+1}, Epoch: {epoch + 1}, Batch: {i + 1}, Loss: {running_loss / 10:.4f}')
              running_loss = 0.0

        input_frames, target_frames = None, None
        frame_dataset = None
        train_loader = None
      os.remove('downloaded_video.mp4')
      counter += 1

print('Finished Training')


Streaming output truncated to the last 5000 lines.
Entering loop
0 / 2998
100 / 2998
200 / 2998
300 / 2998
400 / 2998
500 / 2998
600 / 2998
700 / 2998
800 / 2998
900 / 2998
1000 / 2998
1100 / 2998
1200 / 2998
1300 / 2998
1400 / 2998
1500 / 2998
1600 / 2998
1700 / 2998
1800 / 2998
1900 / 2998
2000 / 2998
2100 / 2998
2200 / 2998
2300 / 2998
2400 / 2998
2500 / 2998
2600 / 2998
2700 / 2998
2800 / 2998
2900 / 2998
Finished loop
Url: 6, Sample: 4, Epoch: 7, Batch: 10, Loss: 0.0002
Url: 6, Sample: 4, Epoch: 7, Batch: 20, Loss: 0.0003
Url: 6, Sample: 4, Epoch: 7, Batch: 30, Loss: 0.0004
Url: 6, Sample: 4, Epoch: 7, Batch: 40, Loss: 0.0002
Url: 6, Sample: 4, Epoch: 7, Batch: 50, Loss: 0.0001
Url: 6, Sample: 4, Epoch: 7, Batch: 60, Loss: 0.0000
Url: 6, Sample: 4, Epoch: 7, Batch: 70, Loss: 0.0001
Url: 6, Sample: 4, Epoch: 7, Batch: 80, Loss: 0.0001
Url: 6, Sample: 4, Epoch: 7, Batch: 90, Loss: 0.0000
Url: 6, Sample: 4, Epoch: 7, Batch: 100, Loss: 0.0001
Url: 6, Sample: 4, Epoch: 7, Batch: 110, L

### Testing the Model

In [ ]:
device = "cuda"
model = model.to(device)

Lets now save the model to another device, just so that we may run it again to other devices

In [ ]:
torch.save(model.state_dict(), 'model_conv_delta1_10epoch.pth')

from google.colab import files

from google.colab import drive
drive.mount('/content/drive')
path = "/content/drive/My Drive/model_conv_delta1_10epoch.pth"
torch.save(model.state_dict(), path)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
